<h1 style='color:purple' align='center'>Data Science Regression Project: Predicting Flats for Rent in Mumbai</h1>

Dataset is downloaded from here: https://www.kaggle.com/jedipro/flats-for-rent-in-mumbai/download

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib 
matplotlib.rcParams["figure.figsize"] = (20,10)

<h2 style='color:blue'>Data Load: Load Flats for Rent in Mumbai into a dataframe</h2>

In [ ]:
df = pd.read_csv('../input/flats-for-rent-in-mumbai/prop_data_clean.csv')
df.head()

In [ ]:
df['city'].unique()

In [ ]:
df.columns

In [ ]:
df.count().sum()

In [ ]:
df['floor_count'].isnull().sum()

**Drop features that are not required to build our model**

In [ ]:
df1 = df.drop(['city','desc','dev_name','floor_count', 'floor_num', 'id', 'id_string','post_date', 'poster_name','project',
       'title', 'trans', 'type', 'url', 'latitude','longitude'],axis='columns')
df1.head()

Area is in Sq feet 

<h2 style='color:blue'>Data Cleaning:</h2>

In [ ]:
df1['user_type'].unique()

In [ ]:
df1.shape

In [ ]:
df1.isnull().sum()

In [ ]:
df2 = df1.dropna()
df2.isnull().sum()

In [ ]:
df2.shape

In [ ]:
df2.head()

In [ ]:
location = df2['locality'].value_counts()
location

In [ ]:
location.values.sum()

In [ ]:
len(location[location>10])

In [ ]:
len(location)

In [ ]:
len(location[location<=10])

In [ ]:
location_stats_less_than_10 = location[location<=10]
location_stats_less_than_10

In [ ]:
len(df2.locality.unique())

In [ ]:
df2['locality'] = df2.locality.apply(lambda x: 'other' if x in location_stats_less_than_10 else x)
len(df2.locality.unique())

In [ ]:
df2.head(10)

In [ ]:
df3 = df2.copy()

In [ ]:
df3.head()

In [ ]:
df3['furnishing'].unique()

In [ ]:
df3.head()

In [ ]:
df3['price_per_sqft'] = df3['price']/df3['area']
df3.head()

In [ ]:
df3_stats = df3['price_per_sqft'].describe()
df3_stats

In [ ]:
df3.to_csv("stage1_sufyan_project_mumbai_home_price.csv",index=False)

In [ ]:
df3.head()

In [ ]:
df3['bedroom_num'].value_counts()

In [ ]:
bathroom_stat = df3['bathroom_num'].value_counts()
bathroom_stat

In [ ]:
bathroom_stat_greater_six = bathroom_stat[bathroom_stat<=4]
bathroom_stat_greater_six

In [ ]:
df3[df3.area/df3.bedroom_num<300].head(20)

In [ ]:
df3.price_per_sqft.describe()

In [ ]:
df3.shape

In [ ]:
df4 = df3.copy()

In [ ]:
df4.head()

In [ ]:
temp = df4[df4.area/df4.bedroom_num<200]
len(temp)

In [ ]:
df4.shape

In [ ]:
df5 = df4[~(df4.area/df4.bedroom_num<200)]
df5.shape

In [ ]:
df5['price_per_sqft'].describe()

In [ ]:
df6 = df5.copy()
df6.head()

In [ ]:
df6.shape

In [ ]:
def remove_pps_outliers(df):
    df_out = pd.DataFrame()
    for key, subdf in df.groupby('locality'):
        m = np.mean(subdf.price_per_sqft)
        st = np.std(subdf.price_per_sqft)
        reduced_df = subdf[(subdf.price_per_sqft>(m-st)) & (subdf.price_per_sqft<=(m+st))]
        df_out = pd.concat([df_out,reduced_df],ignore_index=True)
    return df_out
df7 = remove_pps_outliers(df6)
df7.shape

In [ ]:
df7.head()

In [ ]:
def plot_scatter_chart(df,locality):
    bhk2 = df[(df.locality==locality) & (df.bedroom_num==1)]
    bhk3 = df[(df.locality==locality) & (df.bedroom_num==2)]
    matplotlib.rcParams['figure.figsize'] = (15,10)
    plt.scatter(bhk2.area,bhk2.price,color='blue',label='1 BHK', s=50)
    plt.scatter(bhk3.area,bhk3.price,marker='+', color='green',label='2 BHK', s=50)
    plt.xlabel("Total Square Feet Area")
    plt.ylabel("Price (Lakh Indian Rupees)")
    plt.title(locality)
    plt.legend()
    
plot_scatter_chart(df7,"Andheri East")

In [ ]:
def plot_scatter_chart(df,locality):
    bhk2 = df[(df.locality==locality) & (df.bedroom_num==1)]
    bhk3 = df[(df.locality==locality) & (df.bedroom_num==2)]
    matplotlib.rcParams['figure.figsize'] = (15,10)
    plt.scatter(bhk2.area,bhk2.price,color='blue',label='1 BHK', s=50)
    plt.scatter(bhk3.area,bhk3.price,marker='+', color='green',label='2 BHK', s=50)
    plt.xlabel("Total Square Feet Area")
    plt.ylabel("Price (Lakh Indian Rupees)")
    plt.title(locality)
    plt.legend()
    
plot_scatter_chart(df7,"Lokhandwala Complex")

In [ ]:
df7.shape

In [ ]:
def remove_bhk_outliers(df):
    exclude_indices = np.array([])
    for location, location_df in df.groupby('locality'):
        bhk_stats = {}
        for bhk, bhk_df in location_df.groupby('bedroom_num'):
            bhk_stats[bhk] = {
                'mean': np.mean(bhk_df.price_per_sqft),
                'std': np.std(bhk_df.price_per_sqft),
                'count': bhk_df.shape[0]
            }
        for bhk, bhk_df in location_df.groupby('bedroom_num'):
            stats = bhk_stats.get(bhk-1)
            if stats and stats['count']>5:
                exclude_indices = np.append(exclude_indices, bhk_df[bhk_df.price_per_sqft<(stats['mean'])].index.values)
    return df.drop(exclude_indices,axis='index')
df8 = remove_bhk_outliers(df7)
# df8 = df7.copy()
df8.shape

In [ ]:
def plot_scatter_chart(df,locality):
    bhk2 = df[(df.locality==locality) & (df.bedroom_num==1)]
    bhk3 = df[(df.locality==locality) & (df.bedroom_num==2)]
    matplotlib.rcParams['figure.figsize'] = (15,10)
    plt.scatter(bhk2.area,bhk2.price,color='blue',label='1 BHK', s=50)
    plt.scatter(bhk3.area,bhk3.price,marker='+', color='green',label='2 BHK', s=50)
    plt.xlabel("Total Square Feet Area")
    plt.ylabel("Price (Lakh Indian Rupees)")
    plt.title(locality)
    plt.legend()
    
plot_scatter_chart(df8,"Andheri East")

In [ ]:
def plot_scatter_chart(df,locality):
    bhk2 = df[(df.locality==locality) & (df.bedroom_num==1)]
    bhk3 = df[(df.locality==locality) & (df.bedroom_num==2)]
    matplotlib.rcParams['figure.figsize'] = (15,10)
    plt.scatter(bhk2.area,bhk2.price,color='blue',label='1 BHK', s=50)
    plt.scatter(bhk3.area,bhk3.price,marker='+', color='green',label='2 BHK', s=50)
    plt.xlabel("Total Square Feet Area")
    plt.ylabel("Price (Lakh Indian Rupees)")
    plt.title(locality)
    plt.legend()
    
plot_scatter_chart(df8,"Lokhandwala Complex")

In [ ]:
df9 = df8.copy()
df9.head()

In [ ]:
import matplotlib
matplotlib.rcParams["figure.figsize"] = (20,10)
plt.hist(df9.price_per_sqft,rwidth=0.8)
plt.xlabel("Price Per Square Feet")
plt.ylabel("Count")

In [ ]:
df9['bathroom_num'].unique()

In [ ]:
df9[df9['bathroom_num']>df9['bedroom_num']+2]

In [ ]:
df9 = df9[df9['bathroom_num']<df9['bedroom_num']+2]
df9.shape

In [ ]:
df10 = df9.drop(['price_per_sqft','user_type'],axis='columns')
df10.head()

In [ ]:
df10['furnishing'].unique()

In [ ]:
df11 = df10.copy()

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [ ]:
df11['furnishing']= le.fit_transform(df10['furnishing']) 
df11['furnishing'].unique()

In furnishing column   
 0 = 'Furnished'   
 1 = 'Semi-Furnished'   
 2 = 'Unfurnished'

In [ ]:
df11.head()

In [ ]:
dummies = pd.get_dummies(df11.locality)
dummies.head(3)

In [ ]:
df11 = pd.concat([df11,dummies.drop('other',axis='columns')],axis='columns')
df11.head()

In [ ]:
df12 = df11.drop('locality',axis='columns')
df12.head(2)

In [ ]:
df12.shape

In [ ]:
X = df12.drop(['price'],axis='columns')
X.head(3)

In [ ]:
X.shape

In [ ]:
y = df12.price
y.head(3)

In [ ]:
len(y)

<h2 style='color:blue'>Build a Model Now...</h2>

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=10)

In [ ]:
from sklearn.linear_model import LinearRegression
lr_clf = LinearRegression()
lr_clf.fit(X_train,y_train)
lr_clf.score(X_test,y_test)

K Fold cross validation

In [ ]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)

cross_val_score(LinearRegression(), X, y, cv=cv)

GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor

def find_best_model_using_gridsearchcv(X,y):
    algos = {
        'linear_regression' : {
            'model': LinearRegression(),
            'params': {
                'normalize': [True, False]
            }
        },
        'lasso': {
            'model': Lasso(),
            'params': {
                'alpha': [1,2],
                'selection': ['random', 'cyclic']
            }
        },
        'decision_tree': {
            'model': DecisionTreeRegressor(),
            'params': {
                'criterion' : ['mse','friedman_mse'],
                'splitter': ['best','random']
            }
        }
    }
    scores = []
    cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    for algo_name, config in algos.items():
        gs =  GridSearchCV(config['model'], config['params'], cv=cv, return_train_score=False)
        gs.fit(X,y)
        scores.append({
            'model': algo_name,
            'best_score': gs.best_score_,
            'best_params': gs.best_params_
        })

    return pd.DataFrame(scores,columns=['model','best_score','best_params'])

find_best_model_using_gridsearchcv(X,y)

Test the model for few properties

In [ ]:
def predict_price(location,sqft,bath,bhk):    
    loc_index = np.where(X.columns==location)[0][0]

    x = np.zeros(len(X.columns))
    x[0] = sqft
    x[1] = bath
    x[2] = bhk
    if loc_index >= 0:
        x[loc_index] = 1

    return lr_clf.predict([x])[0]

In [ ]:
predict_price('4 Bunglows',700, 2, 2)

<h2 style='color:blue'>Export the tested model to a pickle file</h2>

In [ ]:
import pickle
with open('banglore_home_prices_model.pickle','wb') as f:
    pickle.dump(lr_clf,f)

<h2 style='color:blue'>Export location and column information to a file</h2>

In [ ]:
import json
columns = {
    'data_columns' : [col.lower() for col in X.columns]
}
with open("columns.json","w") as f:
    f.write(json.dumps(columns))